# Data Preperation

In [1]:
from IPython.display import display, HTML

# Set a maximum height for all output areas
display(HTML('''<style>.output_area pre {max-height: 100px; overflow-y: scroll;}</style>'''))

### Login to HF hub

In [2]:
from huggingface_hub import interpreter_login

# interpreter_login()

## Load Dataset

In [3]:
MY_DATASET = "janaab/supreme-court-speech"

In [4]:
from datasets import load_dataset, DatasetDict

sc_speech = DatasetDict()

sc_speech["train"] = load_dataset(
    MY_DATASET, split="train"
)
sc_speech["test"] = load_dataset(
    MY_DATASET, split="test"
)

print(sc_speech)

Using the latest cached version of the dataset since janaab/supreme-court-speech couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/janaab___supreme-court-speech/default/0.0.0/41c2679acab9b8ecb7f9d881e57d328c67d1e66c (last modified on Fri Aug 30 00:09:29 2024).
Using the latest cached version of the dataset since janaab/supreme-court-speech couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/janaab___supreme-court-speech/default/0.0.0/41c2679acab9b8ecb7f9d881e57d328c67d1e66c (last modified on Fri Aug 30 00:09:29 2024).


DatasetDict({
    train: Dataset({
        features: ['audio', 'transcript', 'duration'],
        num_rows: 10372
    })
    test: Dataset({
        features: ['audio', 'transcript', 'duration'],
        num_rows: 4465
    })
})


In [5]:
sc_speech = sc_speech.select_columns(["audio", "transcript"])

## Feature processing for Whisper

In [6]:
from transformers.models.whisper.tokenization_whisper import TO_LANGUAGE_CODE

TO_LANGUAGE_CODE

{'english': 'en',
 'chinese': 'zh',
 'german': 'de',
 'spanish': 'es',
 'russian': 'ru',
 'korean': 'ko',
 'french': 'fr',
 'japanese': 'ja',
 'portuguese': 'pt',
 'turkish': 'tr',
 'polish': 'pl',
 'catalan': 'ca',
 'dutch': 'nl',
 'arabic': 'ar',
 'swedish': 'sv',
 'italian': 'it',
 'indonesian': 'id',
 'hindi': 'hi',
 'finnish': 'fi',
 'vietnamese': 'vi',
 'hebrew': 'he',
 'ukrainian': 'uk',
 'greek': 'el',
 'malay': 'ms',
 'czech': 'cs',
 'romanian': 'ro',
 'danish': 'da',
 'hungarian': 'hu',
 'tamil': 'ta',
 'norwegian': 'no',
 'thai': 'th',
 'urdu': 'ur',
 'croatian': 'hr',
 'bulgarian': 'bg',
 'lithuanian': 'lt',
 'latin': 'la',
 'maori': 'mi',
 'malayalam': 'ml',
 'welsh': 'cy',
 'slovak': 'sk',
 'telugu': 'te',
 'persian': 'fa',
 'latvian': 'lv',
 'bengali': 'bn',
 'serbian': 'sr',
 'azerbaijani': 'az',
 'slovenian': 'sl',
 'kannada': 'kn',
 'estonian': 'et',
 'macedonian': 'mk',
 'breton': 'br',
 'basque': 'eu',
 'icelandic': 'is',
 'armenian': 'hy',
 'nepali': 'ne',
 'mongol

In [7]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small", language="en", task="transcribe"
)

## Process data

In [8]:
sc_speech["train"].features

{'audio': Audio(sampling_rate=None, mono=True, decode=True, id=None),
 'transcript': Value(dtype='string', id=None)}

In [9]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
sc_speech = sc_speech.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [10]:
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["transcript"],
    )

    # compute input length of audio sample in seconds
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

In [11]:
sc_speech = sc_speech.map(
    prepare_dataset, remove_columns=sc_speech.column_names["train"], num_proc=1
)

In [12]:
max_input_length = 30.0


def is_audio_in_length_range(length):
    return length < max_input_length

In [13]:
sc_speech["train"] = sc_speech["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

In [14]:
sc_speech["train"]

Dataset({
    features: ['input_features', 'labels', 'input_length'],
    num_rows: 10372
})

# Training and Evaluation

## Data Collector

In [15]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [16]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

## Eval metrics

In [17]:
# ! pip install evaluate

In [18]:
import evaluate

metric = evaluate.load("wer")

In [19]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

## Load model checkpoint

In [20]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [21]:
# adjust dropout ratio
model.config.dropout = 0.1

In [22]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language="en", task="transcribe", use_cache=True
)

## Training config

In [23]:
# ! pip install transformers[torch]

In [24]:
MY_MODEL = "janaab/whisper-small-sc"

In [27]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./models/whisper-small-sc",
    push_to_hub=True,
    hub_model_id=MY_MODEL,
    
    # training args
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    lr_scheduler_type="linear",
    warmup_steps=250,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=True,
    # weight_decay=0.01,
    # optim="adamw_bnb_8bit",

    # eval args
    fp16_full_eval=True,
    eval_strategy="steps",
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=250,
    eval_steps=250,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

In [26]:
from transformers import Seq2SeqTrainer, EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Number of evaluations to wait for improvement
    early_stopping_threshold=0.0  # Minimum change to qualify as an improvement
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=sc_speech["train"],
    eval_dataset=sc_speech["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
    callbacks=[early_stopping_callback]
)

/opt/conda/lib/python3.11/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


## Training loop

In [29]:
# # Clearing cache helps with VRAM

# import torch
# torch.cuda.empty_cache()

In [32]:
# # fix AccelerateState bug

# from accelerate import Accelerator
# acc = Accelerator()

In [ ]:
trainer.train()

/opt/conda/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/opt/conda/lib/python3.11/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss,Validation Loss,Wer Ortho,Wer
250,0.279900,0.261353,12.113023,11.626052
500,0.145400,0.236698,11.052283,10.601610


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
/opt/conda/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoi

In [25]:
kwargs = {
    "dataset_tags": MY_DATASET,
    "dataset": "Supreme Court Speech",  # a 'pretty' name for the training dataset
    "language": "en",
    "model_name": "Whisper Small SC - Manas",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [42]:
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/janaab/whisper-small-sc/commit/48f927825ca864d5e91d06619029cbcb4504e0df', commit_message='End of training', commit_description='', oid='48f927825ca864d5e91d06619029cbcb4504e0df', pr_url=None, pr_revision=None, pr_num=None)

In [43]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained("openai/whisper-small")

generation_config.push_to_hub("janaab/whisper-small-sc")

README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/janaab/whisper-small-sc/commit/721126972a216a77e5f91baf618053c47ecb521a', commit_message='Upload config', commit_description='', oid='721126972a216a77e5f91baf618053c47ecb521a', pr_url=None, pr_revision=None, pr_num=None)